# Chapter 11: Path Tracing

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-vision/blob/main/chapter_11_path_tracing.ipynb)

**Path tracing** is a Monte Carlo method for solving the rendering equation that simulates global illumination through recursive random sampling. Unlike traditional ray tracing which only handles direct lighting and perfect reflections, path tracing captures all light transport phenomena including indirect lighting, color bleeding, caustics, and soft shadows.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
from typing import Tuple, List, Optional
from dataclasses import dataclass
from enum import Enum

In [ ]:
class Vec3:
    """3D Vector class"""
    def __init__(self, x: float = 0.0, y: float = 0.0, z: float = 0.0):
        self.x = x
        self.y = y
        self.z = z
    
    def __add__(self, other):
        return Vec3(self.x + other.x, self.y + other.y, self.z + other.z)
    
    def __sub__(self, other):
        return Vec3(self.x - other.x, self.y - other.y, self.z - other.z)
    
    def __mul__(self, scalar: float):
        return Vec3(self.x * scalar, self.y * scalar, self.z * scalar)
    
    def __truediv__(self, scalar: float):
        return Vec3(self.x / scalar, self.y / scalar, self.z / scalar)
    
    def __neg__(self):
        return Vec3(-self.x, -self.y, -self.z)
    
    def dot(self, other) -> float:
        return self.x * other.x + self.y * other.y + self.z * other.z
    
    def cross(self, other):
        return Vec3(
            self.y * other.z - self.z * other.y,
            self.z * other.x - self.x * other.z,
            self.x * other.y - self.y * other.x
        )
    
    def length(self) -> float:
        return math.sqrt(self.dot(self))
    
    def normalize(self):
        l = self.length()
        if l > 0:
            return self / l
        return Vec3(0, 0, 0)
    
    def __repr__(self):
        return f"Vec3({self.x:.3f}, {self.y:.3f}, {self.z:.3f})"

class Color:
    """RGB Color class"""
    def __init__(self, r: float = 0.0, g: float = 0.0, b: float = 0.0):
        self.r = r
        self.g = g
        self.b = b
    
    def __add__(self, other):
        return Color(self.r + other.r, self.g + other.g, self.b + other.b)
    
    def __mul__(self, other):
        if isinstance(other, Color):
            return Color(self.r * other.r, self.g * other.g, self.b * other.b)
        else:
            return Color(self.r * other, self.g * other, self.b * other)
    
    def __truediv__(self, scalar: float):
        return Color(self.r / scalar, self.g / scalar, self.b / scalar)
    
    def clamp(self, min_val: float = 0.0, max_val: float = 1.0):
        return Color(
            max(min_val, min(max_val, self.r)),
            max(min_val, min(max_val, self.g)),
            max(min_val, min(max_val, self.b))
        )
    
    def to_tuple(self) -> Tuple[float, float, float]:
        return (self.r, self.g, self.b)

class Ray:
    """Ray class"""
    def __init__(self, origin: Vec3, direction: Vec3):
        self.origin = origin
        self.direction = direction.normalize()
    
    def at(self, t: float) -> Vec3:
        return self.origin + self.direction * t

print("✓ Base classes loaded")

## 1. The Rendering Equation

**The rendering equation** (Kajiya, 1986) describes how light propagates in a scene:

$$
L_o(\mathbf{p}, \omega_o) = L_e(\mathbf{p}, \omega_o) + \int_{\Omega} f_r(\mathbf{p}, \omega_i, \omega_o) \, L_i(\mathbf{p}, \omega_i) \, (\mathbf{n} \cdot \omega_i) \, d\omega_i
$$

where:
- $L_o(\mathbf{p}, \omega_o)$ = outgoing radiance at point $\mathbf{p}$ in direction $\omega_o$
- $L_e(\mathbf{p}, \omega_o)$ = emitted radiance (from light sources)
- $f_r(\mathbf{p}, \omega_i, \omega_o)$ = BRDF (Bidirectional Reflectance Distribution Function)
- $L_i(\mathbf{p}, \omega_i)$ = incoming radiance from direction $\omega_i$
- $\mathbf{n} \cdot \omega_i$ = cosine term (Lambert's law)
- $\Omega$ = hemisphere around surface normal

### Recursive Formulation

Since $L_i$ depends on $L_o$ at other points, the equation is recursive:

$$
L_o(\mathbf{p}, \omega_o) = L_e(\mathbf{p}, \omega_o) + \int_{\Omega} f_r(\mathbf{p}, \omega_i, \omega_o) \, L_o(\text{raycast}(\mathbf{p}, \omega_i), -\omega_i) \, (\mathbf{n} \cdot \omega_i) \, d\omega_i
$$

This integral has no closed-form solution for general scenes, requiring numerical methods like **Monte Carlo integration**.

## 2. Monte Carlo Integration

**Monte Carlo estimator** for the rendering equation:

$$
\int_{\Omega} f(\omega) \, d\omega \approx \frac{1}{N} \sum_{i=1}^{N} \frac{f(\omega_i)}{p(\omega_i)}
$$

where:
- $\omega_i$ are random samples
- $p(\omega_i)$ is the probability density function (PDF)
- $N$ is the number of samples

### Uniform Hemisphere Sampling

For uniform sampling over hemisphere:

$$
p(\omega) = \frac{1}{2\pi}
$$

The rendering equation becomes:

$$
L_o(\mathbf{p}, \omega_o) \approx L_e(\mathbf{p}, \omega_o) + \frac{2\pi}{N} \sum_{i=1}^{N} f_r(\mathbf{p}, \omega_i, \omega_o) \, L_i(\mathbf{p}, \omega_i) \, (\mathbf{n} \cdot \omega_i)
$$

### Cosine-Weighted Sampling

Better strategy: sample proportional to $\cos\theta$:

$$
p(\omega) = \frac{\cos\theta}{\pi}
$$

This cancels the cosine term in the integral:

$$
L_o(\mathbf{p}, \omega_o) \approx L_e(\mathbf{p}, \omega_o) + \frac{\pi}{N} \sum_{i=1}^{N} f_r(\mathbf{p}, \omega_i, \omega_o) \, L_i(\mathbf{p}, \omega_i)
$$

In [ ]:
def random_in_unit_sphere() -> Vec3:
    """Generate random point in unit sphere"""
    while True:
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
        z = random.uniform(-1, 1)
        if x*x + y*y + z*z < 1:
            return Vec3(x, y, z)

def random_unit_vector() -> Vec3:
    """Generate random unit vector"""
    return random_in_unit_sphere().normalize()

def random_in_hemisphere(normal: Vec3) -> Vec3:
    """Generate random vector in hemisphere around normal"""
    in_unit_sphere = random_in_unit_sphere()
    if in_unit_sphere.dot(normal) > 0.0:
        return in_unit_sphere
    else:
        return -in_unit_sphere

def random_cosine_direction() -> Vec3:
    """Generate random direction with cosine-weighted distribution"""
    r1 = random.random()
    r2 = random.random()
    
    phi = 2.0 * math.pi * r1
    
    x = math.cos(phi) * math.sqrt(r2)
    y = math.sin(phi) * math.sqrt(r2)
    z = math.sqrt(1.0 - r2)
    
    return Vec3(x, y, z)

def create_onb(normal: Vec3) -> Tuple[Vec3, Vec3, Vec3]:
    """Create orthonormal basis from normal vector"""
    w = normal.normalize()
    
    a = Vec3(1, 0, 0) if abs(w.x) < 0.9 else Vec3(0, 1, 0)
    v = w.cross(a).normalize()
    u = w.cross(v)
    
    return (u, v, w)

def local_to_world(local: Vec3, normal: Vec3) -> Vec3:
    """Transform vector from local (tangent) space to world space"""
    u, v, w = create_onb(normal)
    return u * local.x + v * local.y + w * local.z

print("✓ Sampling functions loaded")

## 3. BRDF Models

### Lambertian (Diffuse) BRDF

Perfect diffuse surface scatters light equally in all directions:

$$
f_r = \frac{\rho}{\pi}
$$

where $\rho$ is the albedo (surface color).

### Specular (Mirror) BRDF

Perfect mirror reflection:

$$
\omega_r = \omega_i - 2(\omega_i \cdot \mathbf{n})\mathbf{n}
$$

### Cook-Torrance (Microfacet) BRDF

$$
f_r = \frac{D \, G \, F}{4 (\mathbf{n} \cdot \omega_i) (\mathbf{n} \cdot \omega_o)}
$$

where:
- $D$ = normal distribution function (GGX)
- $G$ = geometry function (shadowing/masking)
- $F$ = Fresnel term

In [ ]:
class MaterialType(Enum):
    DIFFUSE = 1
    METAL = 2
    DIELECTRIC = 3
    EMISSIVE = 4

@dataclass
class Material:
    """Material properties"""
    type: MaterialType
    albedo: Color
    emission: Color = None
    roughness: float = 0.0
    metallic: float = 0.0
    ior: float = 1.5  # Index of refraction
    
    def __post_init__(self):
        if self.emission is None:
            self.emission = Color(0, 0, 0)

def reflect(v: Vec3, n: Vec3) -> Vec3:
    """Reflect vector v about normal n"""
    return v - n * (2.0 * v.dot(n))

def refract(v: Vec3, n: Vec3, eta: float) -> Optional[Vec3]:
    """Refract vector v through interface with normal n"""
    cos_theta = min(-v.dot(n), 1.0)
    r_out_perp = (v + n * cos_theta) * eta
    r_out_parallel_squared = 1.0 - r_out_perp.dot(r_out_perp)
    
    if r_out_parallel_squared < 0:
        return None  # Total internal reflection
    
    r_out_parallel = n * (-math.sqrt(r_out_parallel_squared))
    return r_out_perp + r_out_parallel

def schlick(cosine: float, ref_idx: float) -> float:
    """Schlick's approximation for Fresnel reflectance"""
    r0 = ((1 - ref_idx) / (1 + ref_idx)) ** 2
    return r0 + (1 - r0) * ((1 - cosine) ** 5)

print("✓ BRDF functions loaded")

In [ ]:
@dataclass
class HitRecord:
    """Ray-surface intersection record"""
    t: float
    point: Vec3
    normal: Vec3
    front_face: bool
    material: Material

class Sphere:
    """Sphere primitive"""
    def __init__(self, center: Vec3, radius: float, material: Material):
        self.center = center
        self.radius = radius
        self.material = material
    
    def intersect(self, ray: Ray, t_min: float = 0.001, t_max: float = float('inf')) -> Optional[HitRecord]:
        """Ray-sphere intersection"""
        oc = ray.origin - self.center
        a = ray.direction.dot(ray.direction)
        half_b = oc.dot(ray.direction)
        c = oc.dot(oc) - self.radius * self.radius
        
        discriminant = half_b * half_b - a * c
        if discriminant < 0:
            return None
        
        sqrtd = math.sqrt(discriminant)
        root = (-half_b - sqrtd) / a
        
        if root < t_min or root > t_max:
            root = (-half_b + sqrtd) / a
            if root < t_min or root > t_max:
                return None
        
        t = root
        point = ray.at(t)
        outward_normal = (point - self.center) / self.radius
        front_face = ray.direction.dot(outward_normal) < 0
        normal = outward_normal if front_face else -outward_normal
        
        return HitRecord(
            t=t,
            point=point,
            normal=normal,
            front_face=front_face,
            material=self.material
        )

class Scene:
    """Scene containing objects"""
    def __init__(self):
        self.objects = []
        self.background_color = Color(0.5, 0.7, 1.0)
    
    def add(self, obj):
        self.objects.append(obj)
    
    def intersect(self, ray: Ray, t_min: float = 0.001, t_max: float = float('inf')) -> Optional[HitRecord]:
        """Find nearest intersection"""
        closest_hit = None
        closest_t = t_max
        
        for obj in self.objects:
            hit = obj.intersect(ray, t_min, closest_t)
            if hit:
                closest_t = hit.t
                closest_hit = hit
        
        return closest_hit

print("✓ Scene classes loaded")

## 4. Path Tracing Algorithm

### Basic Path Tracing

```
function trace_path(ray, scene, depth):
    if depth >= max_depth:
        return black
    
    hit = scene.intersect(ray)
    if not hit:
        return background_color
    
    // Emission
    color = hit.material.emission
    
    // Sample random direction
    scatter_direction = sample_hemisphere(hit.normal)
    scatter_ray = Ray(hit.point, scatter_direction)
    
    // Recursive path tracing
    incoming = trace_path(scatter_ray, scene, depth + 1)
    
    // BRDF and cosine term
    brdf = hit.material.albedo / π
    cos_theta = max(0, dot(scatter_direction, hit.normal))
    
    // Monte Carlo estimator
    color += brdf * incoming * cos_theta / pdf
    
    return color
```

### Russian Roulette

To avoid infinite recursion, use **Russian roulette** termination:

$$
P(\text{terminate}) = 1 - \max(\rho_r, \rho_g, \rho_b)
$$

If path continues, boost contribution by $\frac{1}{P(\text{continue})}$ to maintain unbiased estimate.

In [ ]:
def scatter_diffuse(ray: Ray, hit: HitRecord) -> Tuple[Ray, Color, float]:
    """Scatter ray for diffuse material (cosine-weighted)"""
    # Cosine-weighted sampling
    local_dir = random_cosine_direction()
    scatter_direction = local_to_world(local_dir, hit.normal)
    
    # Scattered ray
    scattered = Ray(hit.point, scatter_direction)
    
    # Attenuation (BRDF * cos_theta / pdf)
    # For Lambertian with cosine sampling: albedo/π * cos_theta / (cos_theta/π) = albedo
    attenuation = hit.material.albedo
    
    # PDF value (for reference)
    pdf = scatter_direction.dot(hit.normal) / math.pi
    
    return (scattered, attenuation, pdf)

def scatter_metal(ray: Ray, hit: HitRecord) -> Tuple[Optional[Ray], Color, float]:
    """Scatter ray for metal material"""
    reflected = reflect(ray.direction, hit.normal)
    
    # Add fuzz for non-perfect reflections
    fuzzed = reflected + random_in_unit_sphere() * hit.material.roughness
    scattered = Ray(hit.point, fuzzed)
    
    # Check if scattered ray is in correct hemisphere
    if scattered.direction.dot(hit.normal) > 0:
        return (scattered, hit.material.albedo, 1.0)
    else:
        return (None, Color(0, 0, 0), 0.0)

def scatter_dielectric(ray: Ray, hit: HitRecord) -> Tuple[Ray, Color, float]:
    """Scatter ray for dielectric (glass) material"""
    attenuation = Color(1.0, 1.0, 1.0)
    
    eta = 1.0 / hit.material.ior if hit.front_face else hit.material.ior
    
    unit_direction = ray.direction
    cos_theta = min(-unit_direction.dot(hit.normal), 1.0)
    sin_theta = math.sqrt(1.0 - cos_theta * cos_theta)
    
    cannot_refract = eta * sin_theta > 1.0
    
    # Schlick approximation for reflection probability
    if cannot_refract or schlick(cos_theta, eta) > random.random():
        direction = reflect(unit_direction, hit.normal)
    else:
        direction = refract(unit_direction, hit.normal, eta)
    
    scattered = Ray(hit.point, direction)
    return (scattered, attenuation, 1.0)

def scatter(ray: Ray, hit: HitRecord) -> Tuple[Optional[Ray], Color, float]:
    """Scatter ray based on material type"""
    if hit.material.type == MaterialType.DIFFUSE:
        return scatter_diffuse(ray, hit)
    elif hit.material.type == MaterialType.METAL:
        return scatter_metal(ray, hit)
    elif hit.material.type == MaterialType.DIELECTRIC:
        return scatter_dielectric(ray, hit)
    elif hit.material.type == MaterialType.EMISSIVE:
        return (None, Color(0, 0, 0), 0.0)  # Emissive doesn't scatter
    else:
        return (None, Color(0, 0, 0), 0.0)

print("✓ Scattering functions loaded")

In [ ]:
def trace_path(ray: Ray, scene: Scene, depth: int, max_depth: int = 50) -> Color:
    """Recursive path tracing with Russian roulette"""
    
    # Ray-scene intersection
    hit = scene.intersect(ray)
    
    if not hit:
        # Sky gradient
        t = 0.5 * (ray.direction.y + 1.0)
        return Color(1.0, 1.0, 1.0) * (1.0 - t) + scene.background_color * t
    
    # Emission
    emitted = hit.material.emission
    
    # Scatter ray
    scattered, attenuation, pdf = scatter(ray, hit)
    
    if scattered is None:
        return emitted
    
    # Russian roulette termination (after minimum bounces)
    if depth >= 3:
        max_component = max(attenuation.r, attenuation.g, attenuation.b)
        if random.random() > max_component:
            return emitted
        attenuation = attenuation / max_component
    
    # Hard depth limit
    if depth >= max_depth:
        return emitted
    
    # Recursive path tracing
    incoming = trace_path(scattered, scene, depth + 1, max_depth)
    
    return emitted + attenuation * incoming

print("✓ Path tracing core loaded")

In [ ]:
class Camera:
    """Simple camera"""
    def __init__(self,
                 look_from: Vec3,
                 look_at: Vec3,
                 up: Vec3,
                 vfov: float,
                 aspect_ratio: float):
        
        theta = vfov * math.pi / 180.0
        h = math.tan(theta / 2.0)
        viewport_height = 2.0 * h
        viewport_width = aspect_ratio * viewport_height
        
        self.w = (look_from - look_at).normalize()
        self.u = up.cross(self.w).normalize()
        self.v = self.w.cross(self.u)
        
        self.origin = look_from
        self.horizontal = self.u * viewport_width
        self.vertical = self.v * viewport_height
        self.lower_left_corner = self.origin - self.horizontal / 2.0 - self.vertical / 2.0 - self.w
    
    def get_ray(self, s: float, t: float) -> Ray:
        """Get ray for normalized coordinates (s, t)"""
        direction = self.lower_left_corner + self.horizontal * s + self.vertical * t - self.origin
        return Ray(self.origin, direction)

def render(camera: Camera, scene: Scene, width: int, height: int,
          samples_per_pixel: int = 100, max_depth: int = 50) -> np.ndarray:
    """Render scene using path tracing"""
    image = np.zeros((height, width, 3))
    
    for j in range(height):
        if j % 10 == 0:
            print(f"Scanline {j}/{height}")
        
        for i in range(width):
            color = Color(0, 0, 0)
            
            # Multiple samples per pixel
            for _ in range(samples_per_pixel):
                u = (i + random.random()) / (width - 1)
                v = (j + random.random()) / (height - 1)
                
                ray = camera.get_ray(u, v)
                color = color + trace_path(ray, scene, 0, max_depth)
            
            # Average samples
            color = color / samples_per_pixel
            
            # Gamma correction (gamma = 2.0)
            color = Color(
                math.sqrt(max(0, color.r)),
                math.sqrt(max(0, color.g)),
                math.sqrt(max(0, color.b))
            )
            
            image[height - 1 - j, i] = color.clamp(0.0, 1.0).to_tuple()
    
    return image

print("✓ Renderer loaded")

## Example 1: Cornell Box (Global Illumination)

In [ ]:
# Create Cornell Box scene
scene = Scene()
scene.background_color = Color(0, 0, 0)

# Materials
white_diffuse = Material(MaterialType.DIFFUSE, Color(0.73, 0.73, 0.73))
red_diffuse = Material(MaterialType.DIFFUSE, Color(0.65, 0.05, 0.05))
green_diffuse = Material(MaterialType.DIFFUSE, Color(0.12, 0.45, 0.15))
light_material = Material(MaterialType.EMISSIVE, Color(0.0, 0.0, 0.0), emission=Color(15, 15, 15))

# Walls
scene.add(Sphere(Vec3(0, -1000, 0), 1000, white_diffuse))  # Floor
scene.add(Sphere(Vec3(0, 1005, 0), 1000, white_diffuse))   # Ceiling
scene.add(Sphere(Vec3(0, 0, -1005), 1000, white_diffuse))  # Back wall
scene.add(Sphere(Vec3(-1005, 0, 0), 1000, red_diffuse))    # Left wall (red)
scene.add(Sphere(Vec3(1005, 0, 0), 1000, green_diffuse))   # Right wall (green)

# Objects
scene.add(Sphere(Vec3(-1.5, 1, 1), 1.0, white_diffuse))
scene.add(Sphere(Vec3(1.5, 0.7, 0.5), 0.7, white_diffuse))

# Light source (small sphere on ceiling)
scene.add(Sphere(Vec3(0, 4.8, 0), 0.5, light_material))

# Camera
camera = Camera(
    look_from=Vec3(0, 2.5, 10),
    look_at=Vec3(0, 2.5, 0),
    up=Vec3(0, 1, 0),
    vfov=40,
    aspect_ratio=1.0
)

# Render (low resolution for speed)
print("Rendering Cornell Box...")
image = render(camera, scene, 200, 200, samples_per_pixel=50, max_depth=50)

plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.title('Cornell Box - Global Illumination')
plt.axis('off')
plt.tight_layout()
plt.show()

print("Notice color bleeding: red on left sphere, green on right sphere.")

## Example 2: Mixed Materials (Diffuse, Metal, Glass)

In [ ]:
# Create scene with various materials
scene2 = Scene()
scene2.background_color = Color(0.5, 0.7, 1.0)

# Ground
ground_material = Material(MaterialType.DIFFUSE, Color(0.5, 0.5, 0.5))
scene2.add(Sphere(Vec3(0, -1000, 0), 1000, ground_material))

# Large spheres
diffuse_material = Material(MaterialType.DIFFUSE, Color(0.4, 0.2, 0.1))
scene2.add(Sphere(Vec3(-4, 1, 0), 1.0, diffuse_material))

glass_material = Material(MaterialType.DIELECTRIC, Color(1.0, 1.0, 1.0), ior=1.5)
scene2.add(Sphere(Vec3(0, 1, 0), 1.0, glass_material))

metal_material = Material(MaterialType.METAL, Color(0.7, 0.6, 0.5), roughness=0.0)
scene2.add(Sphere(Vec3(4, 1, 0), 1.0, metal_material))

# Small random spheres
random.seed(42)
for a in range(-3, 3):
    for b in range(-3, 3):
        choose_mat = random.random()
        center = Vec3(a + 0.9 * random.random(), 0.2, b + 0.9 * random.random())
        
        if (center - Vec3(4, 0.2, 0)).length() > 0.9:
            if choose_mat < 0.8:
                # Diffuse
                albedo = Color(random.random() * random.random(),
                             random.random() * random.random(),
                             random.random() * random.random())
                sphere_material = Material(MaterialType.DIFFUSE, albedo)
                scene2.add(Sphere(center, 0.2, sphere_material))
            elif choose_mat < 0.95:
                # Metal
                albedo = Color(0.5 + 0.5 * random.random(),
                             0.5 + 0.5 * random.random(),
                             0.5 + 0.5 * random.random())
                fuzz = random.random() * 0.5
                sphere_material = Material(MaterialType.METAL, albedo, roughness=fuzz)
                scene2.add(Sphere(center, 0.2, sphere_material))
            else:
                # Glass
                sphere_material = Material(MaterialType.DIELECTRIC, Color(1, 1, 1), ior=1.5)
                scene2.add(Sphere(center, 0.2, sphere_material))

# Camera
camera2 = Camera(
    look_from=Vec3(13, 2, 3),
    look_at=Vec3(0, 0, 0),
    up=Vec3(0, 1, 0),
    vfov=20,
    aspect_ratio=16/9
)

# Render
print("Rendering mixed materials scene...")
image2 = render(camera2, scene2, 320, 180, samples_per_pixel=50, max_depth=50)

plt.figure(figsize=(12, 7))
plt.imshow(image2)
plt.title('Path Tracing - Mixed Materials')
plt.axis('off')
plt.tight_layout()
plt.show()

print("Notice realistic reflections, refractions, and global illumination.")

## Summary

**Path tracing** is a physically-based rendering technique that solves the rendering equation using Monte Carlo integration:

### Key Concepts

1. **Rendering Equation**: Describes light transport in the scene
2. **Monte Carlo Integration**: Approximate integrals using random sampling
3. **BRDF**: Bidirectional Reflectance Distribution Function models material appearance
4. **Importance Sampling**: Sample directions proportional to BRDF to reduce variance
5. **Russian Roulette**: Probabilistically terminate paths while maintaining unbiased estimate

### Effects Captured

- **Global illumination**: Indirect lighting from multiple bounces
- **Color bleeding**: Light reflecting colored surfaces tints nearby objects
- **Caustics**: Light focused through reflective/refractive surfaces
- **Soft shadows**: Realistic shadowing from area lights
- **Depth of field**: Camera lens effects

### Algorithm Complexity

- **Time**: $O(\text{pixels} \times \text{samples} \times \text{depth} \times \text{objects})$
- **Convergence**: $\text{Error} \propto \frac{1}{\sqrt{N}}$ where $N$ is sample count
- **Bias**: Unbiased with Russian roulette; biased if using max depth cutoff

### Optimizations

- **Importance sampling**: Sample BRDF lobe instead of uniform hemisphere
- **Next event estimation**: Direct light sampling (explicit light connections)
- **Multiple importance sampling**: Combine sampling strategies
- **Bidirectional path tracing**: Trace from both camera and light

Path tracing is the foundation for physically-based rendering used in:
- Movie production (Pixar, Disney, DreamWorks)
- Architectural visualization
- Product design
- Real-time ray tracing (modern GPUs)